<a href="https://colab.research.google.com/github/jazu1412/LOW_CODE_AUTOML_AUTOGLUON/blob/master/Tabular%20classification%20and%20Regression/Tabular_Indepth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# AutoGluon Tabular - In-Depth Examples

# Setup and Data Loading
# We're preparing our tools and data to predict people's occupations based on their characteristics.
# This is like getting ready to solve a puzzle by organizing all the pieces and understanding what picture we're trying to create.

!pip install autogluon.tabular[all]

from autogluon.tabular import TabularDataset, TabularPredictor
import numpy as np

train_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
subsample_size = 1000
train_data = train_data.sample(n=subsample_size, random_state=0)
print(train_data.head())

label = 'occupation'
print("Summary of occupation column: \n", train_data['occupation'].describe())

test_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
y_test = test_data[label]
test_data_nolabel = test_data.drop(columns=[label])

metric = 'accuracy'


Loaded data from: https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv | Columns = 15 / 15 | Rows = 39073 -> 39073


       age workclass  fnlwgt      education  education-num  \
6118    51   Private   39264   Some-college             10   
23204   58   Private   51662           10th              6   
29590   40   Private  326310   Some-college             10   
18116   37   Private  222450        HS-grad              9   
33964   62   Private  109190      Bachelors             13   

            marital-status        occupation    relationship    race      sex  \
6118    Married-civ-spouse   Exec-managerial            Wife   White   Female   
23204   Married-civ-spouse     Other-service            Wife   White   Female   
29590   Married-civ-spouse      Craft-repair         Husband   White     Male   
18116        Never-married             Sales   Not-in-family   White     Male   
33964   Married-civ-spouse   Exec-managerial         Husband   White     Male   

       capital-gain  capital-loss  hours-per-week  native-country   class  
6118              0             0              40   United-State

Loaded data from: https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv | Columns = 15 / 15 | Rows = 9769 -> 9769


In [5]:

# Hyperparameter Tuning
# We're fine-tuning our prediction models to make them work better for our specific occupation prediction task.
# This is like adjusting the settings on a camera to get the clearest picture possible in different lighting conditions.

from autogluon.common import space

nn_options = {
    'num_epochs': 10,
    'learning_rate': space.Real(1e-4, 1e-2, default=5e-4, log=True),
    'activation': space.Categorical('relu', 'softrelu', 'tanh'),
    'dropout_prob': space.Real(0.0, 0.5, default=0.1),
}

gbm_options = {
    'num_boost_round': 100,
    'num_leaves': space.Int(lower=26, upper=66, default=36),
}

hyperparameters = {
    'GBM': gbm_options,
    'NN_TORCH': nn_options,
}

time_limit = 2*60
num_trials = 5
search_strategy = 'auto'

hyperparameter_tune_kwargs = {
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy,
}

predictor = TabularPredictor(label=label, eval_metric=metric).fit(
    train_data,
    time_limit=time_limit,
    hyperparameters=hyperparameters,
    hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
)


No path specified. Models will be saved in: "AutogluonModels/ag-20240913_195504"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       10.43 GB / 12.67 GB (82.3%)
Disk Space Avail:   65.42 GB / 107.72 GB (60.7%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : F

  0%|          | 0/5 [00:00<?, ?it/s]

Fitted model: LightGBM/T1 ...
	0.37	 = Validation score   (accuracy)
	3.05s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM/T2 ...
	0.355	 = Validation score   (accuracy)
	2.24s	 = Training   runtime
	0.02s	 = Validation runtime
Fitted model: LightGBM/T3 ...
	0.375	 = Validation score   (accuracy)
	1.23s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM/T4 ...
	0.36	 = Validation score   (accuracy)
	1.96s	 = Training   runtime
	0.05s	 = Validation runtime
Fitted model: LightGBM/T5 ...
	0.375	 = Validation score   (accuracy)
	2.08s	 = Training   runtime
	0.02s	 = Validation runtime
Hyperparameter tuning model: NeuralNetTorch ... Tuning model for up to 53.77s of the 107.65s of remaining time.


+---------------------------------------------------+
| Configuration for experiment     NeuralNetTorch   |
+---------------------------------------------------+
| Search algorithm                 SearchGenerator  |
| Scheduler                        FIFOScheduler    |
| Number of trials                 5                |
+---------------------------------------------------+

View detailed results here: /content/AutogluonModels/ag-20240913_195504/models/NeuralNetTorch


Fitted model: NeuralNetTorch/169bee14 ...
	0.355	 = Validation score   (accuracy)
	11.59s	 = Training   runtime
	0.02s	 = Validation runtime
Fitted model: NeuralNetTorch/c03392c8 ...
	0.345	 = Validation score   (accuracy)
	9.22s	 = Training   runtime
	0.08s	 = Validation runtime
Fitted model: NeuralNetTorch/a9bd3a21 ...
	0.315	 = Validation score   (accuracy)
	11.68s	 = Training   runtime
	0.03s	 = Validation runtime
Fitted model: NeuralNetTorch/6d925722 ...
	0.265	 = Validation score   (accuracy)
	6.85s	 = Training   runtime
	0.17s	 = Validation runtime


Fitting model: WeightedEnsemble_L2 ... Training model for up to 119.48s of the 48.8s of remaining time.
	Ensemble Weights: {'LightGBM/T3': 0.955, 'NeuralNetTorch/6d925722': 0.045}
	0.38	 = Validation score   (accuracy)
	0.22s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 71.53s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 1091.3 rows/s (200 batch size)
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20240913_195504")


In [6]:

# Predictions and Evaluation
# Now we're using our fine-tuned model to predict occupations and checking how accurate it is.
# This is like testing our camera settings by taking pictures and seeing how clear they are.

y_pred = predictor.predict(test_data_nolabel)
print("Predictions: ", list(y_pred)[:5])
perf = predictor.evaluate(test_data, auxiliary_metrics=False)


Predictions:  [' Other-service', ' Craft-repair', ' Exec-managerial', ' Sales', ' Other-service']


In [7]:

# Fit Summary
# We're getting an overview of how well our model-tuning process went and what we learned from it.
# This is like reviewing the photos we've taken to understand what settings worked best for different situations.

results = predictor.fit_summary()


*** Summary of fit() ***
Estimated performance of each model:
                     model  score_val eval_metric  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0      WeightedEnsemble_L2      0.380    accuracy       0.183262   8.307901                0.001408           0.220391            2       True         10
1              LightGBM/T3      0.375    accuracy       0.012709   1.233539                0.012709           1.233539            1       True          3
2              LightGBM/T5      0.375    accuracy       0.020838   2.075731                0.020838           2.075731            1       True          5
3              LightGBM/T1      0.370    accuracy       0.009179   3.047611                0.009179           3.047611            1       True          1
4              LightGBM/T4      0.360    accuracy       0.050973   1.963537                0.050973           1.963537            1       True          4
5             

In [8]:

# Model Ensembling
# We're combining multiple prediction models to create a "super model" that's more accurate than any single model.
# This is like asking a group of experts for their opinion instead of relying on just one person's judgment.

label = 'class'
test_data_nolabel = test_data.drop(columns=[label])
y_test = test_data[label]
save_path = 'agModels-predictClass'

predictor = TabularPredictor(label=label, eval_metric=metric).fit(train_data,
    num_bag_folds=5, num_bag_sets=1, num_stack_levels=1,
    hyperparameters = {'NN_TORCH': {'num_epochs': 2}, 'GBM': {'num_boost_round': 20}},
)

predictor = TabularPredictor(label=label, eval_metric='f1', path=save_path).fit(
    train_data, auto_stack=True,
    time_limit=30, hyperparameters={'FASTAI': {'num_epochs': 10}, 'GBM': {'num_boost_round': 200}}
)
predictor.leaderboard(test_data)


No path specified. Models will be saved in: "AutogluonModels/ag-20240913_200012"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       10.14 GB / 12.67 GB (80.0%)
Disk Space Avail:   65.41 GB / 107.72 GB (60.7%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : F

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_BAG_L1,0.629437,0.68559,f1,1.070011,0.150604,34.221982,1.070011,0.150604,34.221982,1,True,1
1,WeightedEnsemble_L2,0.629437,0.68559,f1,1.073305,0.155561,34.234516,0.003295,0.004957,0.012534,2,True,2


from matplotlib import pyplot as plt
_df_0['pred_time_test'].plot(kind='hist', bins=20, title='pred_time_test')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['pred_time_val'].plot(kind='hist', bins=20, title='pred_time_val')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['fit_time'].plot(kind='hist', bins=20, title='fit_time')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_18['model'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_18, x='pred_time_val', y='model', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_19['model'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_19, x='fit_time', y='model', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_20['model'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_20, x='pred_time_test_marginal', y='model', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [9]:

# Decision Threshold Calibration
# We're adjusting how confident our model needs to be before making a prediction, to balance between different types of errors.
# This is like setting the sensitivity of a smoke alarm - too sensitive and it goes off unnecessarily, not sensitive enough and it might miss real fires.

print(f'Prior to calibration (predictor.decision_threshold={predictor.decision_threshold}):')
scores = predictor.evaluate(test_data)

calibrated_decision_threshold = predictor.calibrate_decision_threshold()
predictor.set_decision_threshold(calibrated_decision_threshold)

print(f'After calibration (predictor.decision_threshold={predictor.decision_threshold}):')
scores_calibrated = predictor.evaluate(test_data)

for metric_name in scores:
    metric_score = scores[metric_name]
    metric_score_calibrated = scores_calibrated[metric_name]
    decision_threshold = predictor.decision_threshold
    print(f'decision_threshold={decision_threshold:.3f}\t| metric="{metric_name}"'
          f'\n\ttest_score uncalibrated: {metric_score:.4f}'
          f'\n\ttest_score   calibrated: {metric_score_calibrated:.4f}'
          f'\n\ttest_score        delta: {metric_score_calibrated-metric_score:.4f}')

predictor.set_decision_threshold(0.5)
for metric_name in ['f1', 'balanced_accuracy', 'mcc']:
    metric_score = predictor.evaluate(test_data, silent=True)[metric_name]
    calibrated_decision_threshold = predictor.calibrate_decision_threshold(metric=metric_name, verbose=False)
    metric_score_calibrated = predictor.evaluate(
        test_data, decision_threshold=calibrated_decision_threshold, silent=True
    )[metric_name]
    print(f'decision_threshold={calibrated_decision_threshold:.3f}\t| metric="{metric_name}"'
          f'\n\ttest_score uncalibrated: {metric_score:.4f}'
          f'\n\ttest_score   calibrated: {metric_score_calibrated:.4f}'
          f'\n\ttest_score        delta: {metric_score_calibrated-metric_score:.4f}')


Prior to calibration (predictor.decision_threshold=0.5):


Calibrating decision threshold to optimize metric f1 | Checking 51 thresholds...
Calibrating decision threshold via fine-grained search | Checking 38 thresholds...
	Base Threshold: 0.500	| val: 0.6856
	Best Threshold: 0.500	| val: 0.6856


After calibration (predictor.decision_threshold=0.5):
decision_threshold=0.500	| metric="f1"
	test_score uncalibrated: 0.6294
	test_score   calibrated: 0.6294
	test_score        delta: 0.0000
decision_threshold=0.500	| metric="accuracy"
	test_score uncalibrated: 0.8472
	test_score   calibrated: 0.8472
	test_score        delta: 0.0000
decision_threshold=0.500	| metric="balanced_accuracy"
	test_score uncalibrated: 0.7438
	test_score   calibrated: 0.7438
	test_score        delta: 0.0000
decision_threshold=0.500	| metric="mcc"
	test_score uncalibrated: 0.5457
	test_score   calibrated: 0.5457
	test_score        delta: 0.0000
decision_threshold=0.500	| metric="roc_auc"
	test_score uncalibrated: 0.8990
	test_score   calibrated: 0.8990
	test_score        delta: 0.0000
decision_threshold=0.500	| metric="precision"
	test_score uncalibrated: 0.7411
	test_score   calibrated: 0.7411
	test_score        delta: 0.0000
decision_threshold=0.500	| metric="recall"
	test_score uncalibrated: 0.5470
	test_sc

In [11]:

# Prediction Options
# We're exploring different ways to use our trained model to make predictions about people's occupations.
# This is like learning different techniques to use our camera, like taking single shots, burst mode, or videos, depending on what we're trying to capture.

predictor = TabularPredictor.load(save_path)
print(predictor.features())

datapoint = test_data_nolabel.iloc[[0]]
print(datapoint)
predictor.predict(datapoint)

predictor.predict_proba(datapoint)

print(predictor.model_best)

predictor.leaderboard(test_data)
predictor.leaderboard(extra_info=True)
predictor.leaderboard(test_data, extra_metrics=['accuracy', 'balanced_accuracy', 'log_loss'])

i = 0
model_to_use = predictor.model_names()[i]
model_pred = predictor.predict(datapoint, model=model_to_use)
print("Prediction from %s model: %s" % (model_to_use, model_pred.iloc[0]))

all_models = predictor.model_names()
model_to_use = all_models[i]
specific_model = predictor._trainer.load_model(model_to_use)

model_info = specific_model.get_info()
predictor_information = predictor.info()

y_pred_proba = predictor.predict_proba(test_data_nolabel)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred_proba)

perf = predictor.evaluate(test_data)


['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']
   age workclass  fnlwgt education  education-num       marital-status  \
0   31   Private  169085      11th              7   Married-civ-spouse   

  occupation relationship    race      sex  capital-gain  capital-loss  \
0      Sales         Wife   White   Female             0             0   

   hours-per-week  native-country  
0              20   United-States  
WeightedEnsemble_L2
Prediction from LightGBM_BAG_L1 model:  <=50K


In [12]:

# Interpretability
# We're trying to understand which characteristics are most important in predicting someone's occupation.
# This is like figuring out which ingredients in a recipe contribute most to its taste.

predictor.feature_importance(test_data)




Computing feature importance via permutation shuffling for 14 features using 5000 rows with 5 shuffle sets...
	38.64s	= Expected runtime (7.73s per shuffle set)
	33.71s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
marital-status,0.117047,0.009311,4.765349e-06,5,0.136219,0.097875
capital-gain,0.083102,0.004023,6.569424e-07,5,0.091385,0.074819
education-num,0.071497,0.006756,9.453208e-06,5,0.085407,0.057587
age,0.055806,0.009717,1.059856e-04,5,0.075814,0.035799
occupation,0.055400,0.013509,3.925863e-04,5,0.083214,0.027585
relationship,0.020842,0.006809,1.192225e-03,5,0.034861,0.006822
hours-per-week,0.018326,0.006083,1.264850e-03,5,0.030850,0.005802
capital-loss,0.003712,0.002108,8.495976e-03,5,0.008052,-0.000628
education,0.000719,0.001339,1.480534e-01,5,0.003477,-0.002038
native-country,0.000000,0.000000,5.000000e-01,5,0.000000,0.000000


from matplotlib import pyplot as plt
_df_21['importance'].plot(kind='hist', bins=20, title='importance')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_22['stddev'].plot(kind='hist', bins=20, title='stddev')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_23['p_value'].plot(kind='hist', bins=20, title='p_value')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['n']
  ys = series['p99_high']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_32.sort_values('n', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('n')
_ = plt.ylabel('p99_high')

from matplotlib import pyplot as plt
_df_33['importance'].plot(kind='line', figsize=(8, 4), title='importance')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_34['stddev'].plot(kind='line', figsize=(8, 4), title='stddev')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_35['p_value'].plot(kind='line', figsize=(8, 4), title='p_value')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_36['p99_high'].plot(kind='line', figsize=(8, 4), title='p99_high')
plt.gca().spines[['top', 'right']].set_visible(False)

In [13]:

# Accelerating Inference
# We're speeding up our prediction process so we can handle large amounts of data more quickly.
# This is like upgrading our camera to take photos faster, so we can capture rapid action shots.

predictor.persist()

num_test = 20
preds = np.array(['']*num_test, dtype='object')
for i in range(num_test):
    datapoint = test_data_nolabel.iloc[[i]]
    pred_numpy = predictor.predict(datapoint, as_pandas=False)
    preds[i] = pred_numpy[0]

perf = predictor.evaluate_predictions(y_test[:num_test], preds, auxiliary_metrics=True)
print("Predictions: ", preds)

predictor.unpersist()



Persisting 2 models in memory. Models will require 0.02% of memory.
Unpersisted 2 models: ['WeightedEnsemble_L2', 'LightGBM_BAG_L1']


Predictions:  [' <=50K' ' <=50K' ' >50K' ' <=50K' ' <=50K' ' >50K' ' >50K' ' >50K'
 ' <=50K' ' <=50K' ' <=50K' ' <=50K' ' <=50K' ' <=50K' ' <=50K' ' <=50K'
 ' <=50K' ' >50K' ' >50K' ' <=50K']


['WeightedEnsemble_L2', 'LightGBM_BAG_L1']

In [14]:

# Inference Speed as a Fit Constraint
# We're setting a speed limit for our predictions to ensure they're fast enough for real-time use.
# This is like optimizing our photo-taking process to capture fleeting moments without blur.

infer_limit = 0.00005
infer_limit_batch_size = 10000
predictor_infer_limit = TabularPredictor(label=label, eval_metric=metric).fit(
    train_data=train_data,
    time_limit=30,
    infer_limit=infer_limit,
    infer_limit_batch_size=infer_limit_batch_size,
)

predictor_infer_limit.persist()

predictor_infer_limit.leaderboard()

test_data_batch = test_data.sample(infer_limit_batch_size, replace=True, ignore_index=True)

import time
time_start = time.time()
predictor_infer_limit.predict(test_data_batch)
time_end = time.time()

infer_time_per_row = (time_end - time_start) / len(test_data_batch)
rows_per_second = 1 / infer_time_per_row
infer_time_per_row_ratio = infer_time_per_row / infer_limit
is_constraint_satisfied = infer_time_per_row_ratio <= 1

print(f'Model is able to predict {round(rows_per_second, 1)} rows per second. (User-specified Throughput = {1 / infer_limit})')
print(f'Model uses {round(infer_time_per_row_ratio * 100, 1)}% of infer_limit time per row.')
print(f'Model satisfies inference constraint: {is_constraint_satisfied}')


No path specified. Models will be saved in: "AutogluonModels/ag-20240913_200832"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       9.87 GB / 12.67 GB (77.9%)
Disk Space Avail:   65.40 GB / 107.72 GB (60.7%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fa

Model is able to predict 207883.7 rows per second. (User-specified Throughput = 20000.0)
Model uses 9.6% of infer_limit time per row.
Model satisfies inference constraint: True


In [15]:

# Using Smaller Ensemble or Faster Model for Prediction
# We're creating simplified versions of our model that can make predictions more quickly, trading some accuracy for speed.
# This is like using a smaller, more portable camera that might not have all the features of a professional setup, but is quicker to use.

additional_ensembles = predictor.fit_weighted_ensemble(expand_pareto_frontier=True)
print("Alternative ensembles you can use for prediction:", additional_ensembles)

predictor.leaderboard(only_pareto_frontier=True)

model_for_prediction = additional_ensembles[0]
predictions = predictor.predict(test_data, model=model_for_prediction)
predictor.delete_models(models_to_delete=additional_ensembles, dry_run=False)


Fitting model: WeightedEnsemble_L2Best ...
	Ensemble Weights: {'LightGBM_BAG_L1': 1.0}
	0.6856	 = Validation score   (f1)
	0.01s	 = Training   runtime
	0.0s	 = Validation runtime


Alternative ensembles you can use for prediction: ['WeightedEnsemble_L2Best']


Deleting model WeightedEnsemble_L2Best. All files under agModels-predictClass/models/WeightedEnsemble_L2Best will be removed.


In [16]:

# Collapsing Bagged Ensembles via refit_full
# We're simplifying our complex model into a more streamlined version that's easier to use and understand.
# This is like taking the best aspects of multiple camera setups and combining them into one easy-to-use camera.

refit_model_map = predictor.refit_full()
print("Name of each refit-full model corresponding to a previous bagged ensemble:")
print(refit_model_map)
predictor.leaderboard(test_data)


Refitting models via `predictor.refit_full` using all of the data (combined train and validation)...
	Models trained in this way will have the suffix "_FULL" and have NaN validation score.
	This process is not bound by time_limit, but should take less time than the original `predictor.fit` call.
	To learn more, refer to the `.refit_full` method docstring which explains how "_FULL" models differ from normal models.
Fitting 1 L1 models ...
Fitting model: LightGBM_BAG_L1_FULL ...
	0.82s	 = Training   runtime
Fitting model: WeightedEnsemble_L2_FULL | Skipping fit via cloning parent ...
	Ensemble Weights: {'LightGBM_BAG_L1': 1.0}
	0.01s	 = Training   runtime
Updated best model to "WeightedEnsemble_L2_FULL" (Previously "WeightedEnsemble_L2"). AutoGluon will default to using "WeightedEnsemble_L2_FULL" for predict() and predict_proba().
Refit complete, total runtime = 0.93s ... Best model: "WeightedEnsemble_L2_FULL"


Name of each refit-full model corresponding to a previous bagged ensemble:
{'LightGBM_BAG_L1': 'LightGBM_BAG_L1_FULL', 'WeightedEnsemble_L2': 'WeightedEnsemble_L2_FULL'}


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_BAG_L1_FULL,0.634860,NaN,f1,0.123911,NaN,0.818707,0.123911,NaN,0.818707,1,True,3
1,WeightedEnsemble_L2_FULL,0.634860,NaN,f1,0.130020,NaN,0.831241,0.006109,NaN,0.012534,2,True,4
2,LightGBM_BAG_L1,0.629437,0.68559,f1,1.144639,0.150604,34.221982,1.144639,0.150604,34.221982,1,True,1
3,WeightedEnsemble_L2,0.629437,0.68559,f1,1.147901,0.155561,34.234516,0.003262,0.004957,0.012534,2,True,2


In [17]:

# Model Distillation
# We're creating a simplified version of our best model that's almost as good but much faster and easier to use.
# This is like creating a point-and-shoot camera that captures photos almost as well as a professional setup, but is much simpler to operate.

student_models = predictor.distill(time_limit=30)
print(student_models)
preds_student = predictor.predict(test_data_nolabel, model=student_models[0])
print(f"predictions from {student_models[0]}:", list(preds_student)[:5])
predictor.leaderboard(test_data)


Distilling with teacher='WeightedEnsemble_L2_FULL', teacher_preds=soft, augment_method=spunge ...
SPUNGE: Augmenting training data with 4000 synthetic samples for distillation...
Distilling with each of these student models: ['LightGBM_DSTL', 'CatBoost_DSTL', 'RandomForestMSE_DSTL', 'NeuralNetTorch_DSTL']
Fitting 4 L1 models ...
Fitting model: LightGBM_DSTL ... Training model for up to 30.0s of the 29.99s of remaining time.
		pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: workclass: object, education: object, marital-status: object, occupation: object, relationship: object, race: object, native-country: object
Detailed Traceback:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/autogluon/core/trainer/abstract_trainer.py", line 1904, in _train_and_save
    model = self._train_single(X, y, model, X_val, y_val, total_resources=total_resources, **model_fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/autogluon/core/trai

['RandomForestMSE_DSTL', 'WeightedEnsemble_L2_DSTL']
predictions from RandomForestMSE_DSTL: [' <=50K', ' <=50K', ' >50K', ' <=50K', ' <=50K']


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_BAG_L1_FULL,0.634860,NaN,f1,0.124121,NaN,0.818707,0.124121,NaN,0.818707,1,True,3
1,WeightedEnsemble_L2_FULL,0.634860,NaN,f1,0.127171,NaN,0.831241,0.003049,NaN,0.012534,2,True,4
2,LightGBM_BAG_L1,0.629437,0.685590,f1,1.054768,0.150604,34.221982,1.054768,0.150604,34.221982,1,True,1
3,WeightedEnsemble_L2,0.629437,0.685590,f1,1.058058,0.155561,34.234516,0.003291,0.004957,0.012534,2,True,2
4,RandomForestMSE_DSTL,0.607631,0.629213,mean_squared_error,1.231080,0.159267,16.055524,1.231080,0.159267,16.055524,1,True,5
5,WeightedEnsemble_L2_DSTL,0.607631,0.629213,mean_squared_error,1.234787,0.160522,16.062659,0.003707,0.001255,0.007135,2,True,6


In [18]:

# Faster Presets or Hyperparameters
# We're using pre-configured settings to quickly create models that are good enough for many purposes without requiring much setup time.
# This is like using the automatic mode on a camera - it might not give you the absolute best picture possible, but it's quick and easy to use.

presets = ['good_quality', 'optimize_for_deployment']
predictor_light = TabularPredictor(label=label, eval_metric=metric).fit(train_data, presets=presets, time_limit=60)

predictor_light = TabularPredictor(label=label, eval_metric=metric).fit(train_data, hyperparameters='very_light', time_limit=60)

excluded_model_types = ['KNN', 'NN_TORCH']
predictor_light = TabularPredictor(label=label, eval_metric=metric).fit(train_data, excluded_model_types=excluded_model_types, time_limit=60)

print("AutoGluon Tabular In-Depth Examples Completed")

No path specified. Models will be saved in: "AutogluonModels/ag-20240913_201339"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       9.81 GB / 12.67 GB (77.4%)
Disk Space Avail:   65.37 GB / 107.72 GB (60.7%)
Presets specified: ['good_quality', 'optimize_for_deployment']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Note: `save_bag_folds=False`! This will greatly reduce peak disk usage during fit (by ~8x), but runs the risk of an out-of-memory error during model refit if memory is small relative to the data size.
	You can avoid this risk by setting `save_bag_f

AutoGluon Tabular In-Depth Examples Completed
